In [13]:
import pandas as pd

In [14]:
from owlready2 import *

In [15]:
test = False
if test:
    import owlready2
    print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

In [16]:
onto = get_ontology("pbn_t3_5.owl").load()

In [20]:
for k in onto.classes():
    K = str(k).split(".")[-1]
    #print("## ",K," -- ",len(k.instances()), "instances.")
    for i in k.instances()[:5]: 
        I = str(i).split(".")[-1]
        for p in list(i.get_properties()):
            P = str(p).split(".")[-1]
            OBJ = "onto."+str(I)+"."+str(P)
            #print(OBJ,P)
            try:
                EV = eval(OBJ)
                INV = eval("onto."+str(P)+".inverse")
                if INV:
                    INVERSE = str(INV).split(".")[-1]
                    print(OBJ,INVERSE)
            except:
                pass#print(eval(OBJ))
            #if not P == "has_str":
            #    print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",eval(OBJ))

onto.PBN__Benef_0.has_BenefSource has_BenefInArticle
onto.PBN__Benef_1.has_BenefSource has_BenefInArticle
onto.PBN__Benef_2.has_BenefSource has_BenefInArticle
onto.PBN__Benef_3.has_BenefSource has_BenefInArticle
onto.PBN__Benef_4.has_BenefSource has_BenefInArticle
onto.PBN__Risk_0.has_RiskMitigation has_MitigationRisk
onto.PBN__Risk_1.has_RiskMitigation has_MitigationRisk
onto.PBN__Risk_2.has_RiskMitigation has_MitigationRisk
onto.PBN__Risk_3.has_RiskMitigation has_MitigationRisk
onto.PBN__Risk_4.has_RiskMitigation has_MitigationRisk
onto.PBN__Article_0.has_BenefInArticle has_BenefSource
onto.PBN__Article_0.has_RiskInArticle has_RiskSource
onto.PBN__Article_24.has_BenefInArticle has_BenefSource
onto.PBN__Article_24.has_RiskInArticle has_RiskSource
onto.PBN__Article_118.has_BenefInArticle has_BenefSource
onto.PBN__Article_118.has_RiskInArticle has_RiskSource
onto.PBN__Article_204.has_BenefInArticle has_BenefSource
onto.PBN__Article_204.has_RiskInArticle has_RiskSource
onto.PBN__Article_

In [21]:
print("\n# Overview\n")
for k in onto.classes():
    if len(k.instances()):
        print("* ","_"+str(k).split(".")[-1]+"_"," -- ",len(k.instances()), "instances.")
print("\n# Details\n")
for k in onto.classes():
    K = str(k).split(".")[-1]
    print("## ",K," -- ",len(k.instances()), "instances.")
    for i in k.instances()[:3]: 
        I = str(i).split(".")[-1]
        if K == "Qid":
            if len(i.has_Qid_alias) > 0:
                print("### ",I," -> ",i.has_Qid_alias[0])
            else:
                print("### ",I)
            for p in list(i.get_properties()):
                P = str(p).split(".")[-1]
                OBJ = "onto."+str(I)+"."+str(P)
                if not P == "has_str":
                    print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",eval(OBJ))
        else:
            print("### ",I," -> ",i.has_str[0])
            for p in list(i.get_properties()):
                P = str(p).split(".")[-1]
                OBJ = "onto."+str(I)+"."+str(P)
                if not P == "has_str":
                    print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",eval(OBJ))



# Overview

*  _Benef_  --  1415 instances.
*  _BenefReturn_  --  1606 instances.
*  _Risk_  --  3471 instances.
*  _Article_  --  376 instances.
*  _RiskGBN_  --  5 instances.
*  _RiskMitigation_  --  5606 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2908 instances.
*  _Qid_  --  1728 instances.
*  _Technology_  --  4738 instances.
*  _RiskISO_Impact_  --  24 instances.
*  _RiskISO_Purpose_  --  19 instances.
*  _StakeholderGroup_  --  184 instances.
*  _StakeholderSubgroup_  --  169 instances.
*  _TechGroup_  --  210 instances.
*  _TechSubgroup_  --  191 instances.
*  _Blueprint_  --  24 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Intervention_  --  73 instances.
*  _Mitigation_  --  2485 instances.
*  _PBNThing_  --  20314 instances.
*  _RiskISO_  --  43 instances.

# Details

##  Benef  --  1415 instances.
###  PBN__Benef_0  ->  Early detection
*  pbn_t3_5.PBN__Benef_0.has_Beneficiary

In [22]:
for k in onto.classes():
    if len(k.instances()):
        print("* ","_"+str(k).split(".")[-1]+"_"," -- ",len(k.instances()), "instances.")

*  _Benef_  --  1415 instances.
*  _BenefReturn_  --  1606 instances.
*  _Risk_  --  3471 instances.
*  _Article_  --  376 instances.
*  _RiskGBN_  --  5 instances.
*  _RiskMitigation_  --  5606 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2908 instances.
*  _Qid_  --  1728 instances.
*  _Technology_  --  4738 instances.
*  _RiskISO_Impact_  --  24 instances.
*  _RiskISO_Purpose_  --  19 instances.
*  _StakeholderGroup_  --  184 instances.
*  _StakeholderSubgroup_  --  169 instances.
*  _TechGroup_  --  210 instances.
*  _TechSubgroup_  --  191 instances.
*  _Blueprint_  --  24 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Intervention_  --  73 instances.
*  _Mitigation_  --  2485 instances.
*  _PBNThing_  --  20314 instances.
*  _RiskISO_  --  43 instances.


# Doing risks

In [34]:
import os

In [23]:
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
for k in onto.classes():
    K = str(k).split(".")[-1]
    if K not in ["Qid","RiskMitigation","PBNThing"]:
        PATH = "./docs/"+K+"/"
        #PATH = '7.pages/'+K+"/"
        if not os.path.isdir(PATH):
            os.mkdir(PATH)
        #print("## ",K," -- ",len(k.instances()), "instances.")
        for i in k.instances(): 
            I = str(i).split(".")[-1]
            MD = "[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md) -- ["+K+"](https://github.com/mm80843/T3.5/tree/main/docs/"+K+"/index.md) \n\n"
            MD += "# "+K+": __"+i.has_str[0]+"__ ("+i.get_name()+")\n\n"
            FILENAME = PATH+i.get_name()+".md"
            if 1:#not os.path.isfile(FILENAME):
                #print("### ",I," -> ",i.has_str[0])
                for p in list(i.get_properties()):
                    P = str(p).split(".")[-1]
                    OBJ = "onto."+str(I)+"."+str(P)
                    OB = eval(OBJ)
                    if not (OB == [onto.nan]):
                        if not P == "has_str":
                            MD += "## Property: "+P+"\n\n"
                            #print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",[(x,x.has_str[0]) for x in eval(OBJ)])
                            #print(P,eval(OBJ))

                            if type(eval(OBJ)[0]) is str:
                                MD += eval(OBJ)[0] + "\n\n"
                            else:

                                A = [x.get_name() for x in eval(OBJ)]
                                if "_" in str(A[0]):
                                    A = [LINK+x.split("PBN__")[1].split("_")[0]+"/"+x+".md" for x in A]
                                    #print(str(A[0]),A)
                                else:                                
                                    A = [LINK+"/Qid/"+x+".md" for x in A]
                                #print(A,eval(OBJ))
                                if (OB[0].is_instance_of[0] == onto.Qid):
                                    B = [x.get_name() for x in eval(OBJ)]
                                else:
                                    B = [x.has_str[0] for x in eval(OBJ)]
                                for k in range(len(A)):
                                    MD += "* ["+B[k]+"]("+A[k]+")\n"
                                MD += "\n"
                #print(MD)
                with open(FILENAME,"w",encoding="utf-8") as f:
                    f.write(MD)

In [24]:
print(MD)

[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md) -- [RiskISO](https://github.com/mm80843/T3.5/tree/main/docs/RiskISO/index.md) 

# RiskISO: __Environment and sustainable production and consumption__ (PBN__RiskISO_Purpose_18)




In [25]:
IDX = ""
IDX += "# Overview\n\n"
for k in onto.classes():
    PATH =  "./docs/"
    #PATH = "7.pages/" 
    if len(k.instances()):
        MM = str(k).split(".")[-1]
        if (not MM == "RiskMitigation") and (not MM == "PBNThing"):
            IDX += "* _["+MM+"]("+LINK+MM+"/index.md)_ -- "+str(len(k.instances()))+ " instances.\n"
with open(PATH+"index.md","w",encoding="utf-8") as f:
        f.write(IDX)

In [26]:
print(IDX)

# Overview

* _[Benef](https://github.com/mm80843/T3.5/blob/main/docs/Benef/index.md)_ -- 1415 instances.
* _[BenefReturn](https://github.com/mm80843/T3.5/blob/main/docs/BenefReturn/index.md)_ -- 1606 instances.
* _[Risk](https://github.com/mm80843/T3.5/blob/main/docs/Risk/index.md)_ -- 3471 instances.
* _[Article](https://github.com/mm80843/T3.5/blob/main/docs/Article/index.md)_ -- 376 instances.
* _[RiskGBN](https://github.com/mm80843/T3.5/blob/main/docs/RiskGBN/index.md)_ -- 5 instances.
* _[RiskHealth](https://github.com/mm80843/T3.5/blob/main/docs/RiskHealth/index.md)_ -- 14 instances.
* _[RiskType](https://github.com/mm80843/T3.5/blob/main/docs/RiskType/index.md)_ -- 22 instances.
* _[Stakeholder](https://github.com/mm80843/T3.5/blob/main/docs/Stakeholder/index.md)_ -- 2908 instances.
* _[Qid](https://github.com/mm80843/T3.5/blob/main/docs/Qid/index.md)_ -- 1728 instances.
* _[Technology](https://github.com/mm80843/T3.5/blob/main/docs/Technology/index.md)_ -- 4738 instances.
* _[

In [27]:
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
for k in onto.classes():
    K = str(k).split(".")[-1]
    if len(k.instances()):
    #print(K)
        PATH =  "./docs/"
        #PATH = "7.pages/" 
        if K not in ["RiskMitigation","PBNThing","Qid"]:
            IDX = "[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md)"
            IDX += "\n\n# "+str(K)+" -- "+str(len(k.instances()))+ " instances.\n\n"
            for i in k.instances(): 
                I = str(i).split(".")[-1]
                if K == "Qid":
                    if len(i.has_Qid_alias) > 0:

                        IDX += "* ["+i.has_Qid_alias[0]+"]("+LINK+"Qid/"+str(I)+".md)\n"
                    else:
                        IDX += "* ["+str(I)+"]\n"
                else:
                    II = I.split("PBN__")[1].split("_")[0]
                    A = i.has_str[0]
                    if K == "Article":
                        A = " ".join(list(set(A.split(" ")))).replace("nan","").strip()
                    IDX += "* ["+A+"]("+LINK+II+"/"+str(I)+".md)\n" 
            with open(PATH+K+"/index.md","w",encoding="utf-8") as f:
                f.write(IDX)

In [28]:
print(IDX)

[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md)

# RiskISO -- 43 instances.

* [Resilience](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_2.md)
* [Health and care](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_3.md)
* [Responsible resource use](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_4.md)
* [Well-being](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_1.md)
* [Preserv. and improve. of the environment](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_7.md)
* [Topic](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_12.md)
* [Economy and sustainable production and consumption](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_10.md)
* [Safety and security](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_0.md)
* [Preservation and improvement of the env